# Healthcare-AI Deployment Notebook
Run this notebook in Google Colab to deploy the AI Agent.

In [ ]:
# 0. Check GPU Status
!nvidia-smi
# If this fails, go to Runtime -> Change runtime type -> T4 GPU

In [ ]:
# 1. Install System Dependencies & Ollama
!curl -fsSL https://ollama.com/install.sh | sh
!apt-get install -y libduckdb-dev # Optional depending on duckdb wheel availability

# Install lshw to help Ollama detect GPU if needed
!apt-get install -y pciutils lshw

In [ ]:
# 2. Start Ollama Server in Background
import subprocess
import time
import urllib.request
import socket
import os

# Set LD_LIBRARY_PATH to help Ollama find NVIDIA libraries
os.environ["LD_LIBRARY_PATH"] = "/usr/lib64-nvidia:/usr/local/nvidia/lib:/usr/local/nvidia/lib64"

# Start Ollama and redirect logs to a file for debugging
with open("ollama.log", "w") as log_file:
    process = subprocess.Popen(['ollama', 'serve'], stdout=log_file, stderr=subprocess.STDOUT)

print("Starting Ollama...")

# Poll for service availability
start_time = time.time()
while time.time() - start_time < 60:
    try:
        with urllib.request.urlopen("http://localhost:11434") as response:
            if response.status == 200:
                print("Ollama is running!")
                break
    except (urllib.error.URLError, ConnectionRefusedError, socket.error):
        time.sleep(1)
else:
    print("Error: Ollama failed to start within 60 seconds.")
    print("--- Ollama Logs ---")
    with open("ollama.log", "r") as f:
        print(f.read())

In [ ]:
# 3. Pull Typhoon Models (This may take a few minutes)
!ollama pull scb10x/llama3.1-typhoon2-8b-instruct:latest
!ollama pull scb10x/llama3.2-typhoon2-1b-instruct:latest
!ollama list

In [ ]:
# 4. Fetch Code from GitHub (CI/CD)
import os

REPO_URL = "https://github.com/pongchanan/Healthcare-AI.git"
PROJECT_DIR = "Healthcare-AI"

if os.path.exists(PROJECT_DIR):
    print(f"Updating existing repository in {PROJECT_DIR}...")
    %cd {PROJECT_DIR}
    !git pull
    %cd ..
else:
    print(f"Cloning repository from {REPO_URL}...")
    !git clone {REPO_URL}

print("✅ Code synced successfully!")

In [ ]:
# 5. Install Python Dependencies
%cd Healthcare-AI
!pip install -r requirements.txt
!pip install pyngrok # For exposing the server

In [ ]:
# 6. Run Ingestion (Build Databases)
!python -m src.pipelines.sql_loader
!python -m src.pipelines.ingestion

In [ ]:
# 7. Start the API with Ngrok Public URL
from pyngrok import ngrok
import nest_asyncio
import uvicorn
from main import app

# Authenticate ngrok (Optional but recommended for stability - get token from ngrok.com)
ngrok.set_auth_token("33KFiXOGZWUksDWun4lAmtyfPE6_DCTBB1Dq4f2pwiZtfNXz")

public_url = ngrok.connect(8000)
print(f"\n🔥 Public API URL: {public_url}\n")

nest_asyncio.apply()
# uvicorn.run(app, host="0.0.0.0", port=8000)

# In Colab/Jupyter, we must run uvicorn asynchronously to avoid "asyncio.run() cannot be called from a running event loop"
config = uvicorn.Config(app, host="0.0.0.0", port=8000)
server = uvicorn.Server(config)
await server.serve()

In [ ]:
# DEBUG: Check Ollama Logs if issues persist
!cat /content/ollama.log